<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [43]:
!pip3 install beautifulsoup4
!pip3 install requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
wiki_url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
html_data = requests.get(wiki_url).text
soup = BeautifulSoup(html_data, "html.parser")

!pip3 install beautifulsoup4 requests


In [46]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd


and we will provide some helper functions for you to process web scraped HTML table


In [50]:
def date_time(table_cell):
    """
    Return (date, time) extracted from the first table cell of a row.
    Safe: returns ('', '') if nothing found.
    """
    try:
        texts = [t.strip() for t in table_cell.strings if t.strip()]
        if len(texts) >= 2:
            return texts[0], texts[1]
        elif len(texts) == 1:
            # sometimes date and time appear combined or separated by newline
            parts = texts[0].split()
            # fallback: return whole text as date, empty time
            return texts[0], ''
        else:
            return '', ''
    except Exception:
        return '', ''

def booster_version(table_cell):
    """
    Return booster version text from its cell.
    We remove <sup>, <a>, <small> tags to avoid footnote text.
    """
    try:
        # remove tags that pollute the text
        for t in table_cell.find_all(['sup','small']):
            t.extract()
        txt = table_cell.get_text(" ", strip=True)
        return txt
    except Exception:
        return ''

def landing_status(table_cell):
    """Return first meaningful text from booster landing cell."""
    try:
        txt = table_cell.get_text(" ", strip=True)
        return txt
    except Exception:
        return ''

def get_mass(table_cell):
    """
    Extract numeric mass in kg as a string without 'kg' (e.g. "5420").
    Return '' if not found.
    """
    try:
        s = unicodedata.normalize("NFKD", table_cell.get_text(" ", strip=True))
        m = re.search(r'([0-9\.,]+)\s*kg', s)
        if m:
            return m.group(1).replace(',', '')
        else:
            return ''
    except Exception:
        return ''

def extract_column_from_header(th_tag):
    """
    Extract a clean column header from a <th> element.
    Returns None for empty/digit-only headers.
    """
    try:
        # remove tags that create noise
        for tag in th_tag.find_all(['sup','a','br']):
            tag.extract()
        name = th_tag.get_text(" ", strip=True)
        if name and not name.strip().isdigit():
            return name.strip()
        return None
    except Exception:
        return None

To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [53]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}


Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [56]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object
wiki_url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
html_data = requests.get(wiki_url).text
soup = BeautifulSoup(html_data, "html.parser")

resp = requests.get(static_url, headers=headers, timeout=15)
if resp.status_code != 200:
    raise RuntimeError(f"Request failed: status code {resp.status_code}")
html_data = resp.text
soup = BeautifulSoup(html_data, "html.parser")

Create a `BeautifulSoup` object from the HTML `response`


In [25]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content



Print the page title to verify if the `BeautifulSoup` object was created properly 


In [55]:
# Use soup.title attribute
print("Page title:", soup.title.string.strip())

Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [32]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`


Starting from the third table is our target table contains the actual launch records.


In [59]:
# Let's print the third table and check its content


html_tables = soup.find_all('table', class_='wikitable plainrowheaders collapsible')
print("Number of wikitable plainrowheaders collapsible tables found:", len(html_tables))

# inspect the 3rd table (index 2)
first_launch_table = html_tables[2]

# extract column names from its <th> elements
column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print("Extracted column names (example):", column_names)



Number of wikitable plainrowheaders collapsible tables found: 9
Extracted column names (example): ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [61]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

launch_dict = {
    'Flight No.': [],
    'Date': [],
    'Time': [],
    'Version Booster': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Booster landing': []
}


Check the extracted column names


In [62]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [63]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column


# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

extracted_row = 0

for table_number, table in enumerate(html_tables):
    for rows in table.find_all("tr"):
        th = rows.find('th')
        flight_number = None
        if th:
            text = th.get_text(strip=True)
            # sometimes there are additional characters; keep only pure digits as flights
            if text and text.split()[0].isdigit():
                flight_number = text.split()[0]

        if flight_number:
            tds = rows.find_all('td')
            # ensure we have enough cells (some rows are headings or notes)
            if len(tds) < 9:
                # skip unexpected row layout
                continue

            extracted_row += 1
            # Flight No.
            launch_dict['Flight No.'].append(flight_number)

            # Date & Time
            d, tm = date_time(tds[0])
            launch_dict['Date'].append(d.strip(','))
            launch_dict['Time'].append(tm)

            # Version Booster
            bv = booster_version(tds[1])
            if not bv:
                bv = tds[1].get_text(" ", strip=True)
            launch_dict['Version Booster'].append(bv)

            # Launch site
            ls = tds[2].get_text(" ", strip=True)
            launch_dict['Launch site'].append(ls)

            # Payload
            payload = tds[3].get_text(" ", strip=True)
            launch_dict['Payload'].append(payload)

            # Payload mass (string; we'll convert to numeric later)
            pm = get_mass(tds[4])
            launch_dict['Payload mass'].append(pm if pm else None)

            # Orbit
            orbit = tds[5].get_text(" ", strip=True)
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = tds[6].get_text(" ", strip=True)
            launch_dict['Customer'].append(customer)

            # Launch outcome
            launch_outcome = tds[7].get_text(" ", strip=True)
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = tds[8].get_text(" ", strip=True)
            launch_dict['Booster landing'].append(booster_landing)

print("Rows extracted:", extracted_row)


Rows extracted: 121


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [65]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            
            df = pd.DataFrame(launch_dict)

# Convert payload mass to numeric (kg)
df['Payload mass'] = pd.to_numeric(df['Payload mass'], errors='coerce')

# How many missing before filling?
missing_before = df['Payload mass'].isna().sum()
print("Missing payload mass before fill:", missing_before)

# Fill missing payload mass with mean (if any)
mean_mass = df['Payload mass'].mean()
df['Payload mass'].fillna(mean_mass, inplace=True)

missing_after = df['Payload mass'].isna().sum()
print("Missing payload mass after fill:", missing_after)

# Basic checks
print("DataFrame shape:", df.shape)
print(df.head(8))

# Export to CSV for later labs
df.to_csv('spacex_web_scraped.csv', index=False)
print("Saved spacex_web_scraped.csv")

            

F9 v1.0 B0003.1
F9 v1.0 B0004.1
F9 v1.0 B0005.1
F9 v1.0 B0006.1
F9 v1.0 B0007.1
F9 v1.1 B1003
F9 v1.1 B1004
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1 B1011
F9 v1.1 B1010
F9 v1.1 B1012
F9 v1.1 B1013
F9 v1.1 B1014
F9 v1.1 B1015
F9 v1.1 B1016
F9 v1.1 B1018
F9 FT B1019.1
F9 v1.1 B1017
F9 FT B1020.1
F9 FT B1021.1
F9 FT B1022.1
F9 FT B1023.1
F9 FT B1024.1
F9 FT B1025.1
F9 FT B1026.1
F9 FT B1029.1
F9 FT B1031.1
F9 FT B1030.1
F9 FT ♺ B1021.2
F9 FT B1032.1
F9 FT B1034.1
F9 FT B1035.1
F9 FT ♺ B1029.2
F9 FT B1036.1
F9 FT B1037.1
F9 B4 B1039.1
F9 FT B1038.1
F9 B4 B1040.1
F9 B4 B1041.1
F9 FT ♺ B1031.2
F9 B4 B1042.1
F9 FT ♺ B1035.2
F9 FT ♺ B1036.2
F9 B4 B1043.1
F9 FT ♺ B1032.2
F9 FT ♺ B1038.2
F9 B4 B1044.1
F9 B4 ♺ B1041.2
F9 B4 ♺ B1039.2
F9 B4 B1045.1
F9 B5 B1046.1
F9 B4 ♺ B1043.2
F9 B4 ♺ B1040.2
F9 B4 ♺ B1045.2
F9 B5 B1047.1
F9 B5 B1048 .1
F9 B5 ♺ B1046.2
F9 B5 B1049.1
F9 B5 ♺ B1048.2
F9 B5 ♺ B1047.2
F9 B5 ♺ B1046.3 SHERPA
F9 B5 B1050
F9 B5 B1054
F9 B5 ♺ B1049.2
F9 B5 ♺ B1048.3
F9 B5 B1051.1
F9 B5 B1

AttributeError: 'NoneType' object has no attribute 'string'

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [67]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
# 1) Title check (we did earlier)
print("Title:", soup.title.string.strip())

# 2) column names
print("Columns:", df.columns.tolist())

# 3) counts
print("Total launches parsed:", len(df))

# 4) sample landing column values distribution
print(df['Booster landing'].value_counts().head(10))


Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Columns: ['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time']
Total launches parsed: 121
Success (drone ship)                53
Success (ground pad)                14
Failure (drone ship)                 6
No attempt [ 165 ]                   6
No attempt                           5
Controlled (ocean) [ d ] [ 46 ]      2
Failure [ 383 ] (ground pad)         1
Success [ 315 ] (drone ship)         1
Controlled (ocean) [ d ] [ 248 ]     1
No attempt [ 271 ]                   1
Name: Booster landing, dtype: int64


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
